<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import tensorflow as tf
import numpy as np
print('TensorFlow Version :',tf.__version__)
import os
import sys
print('Python interpreter and version:',sys.version)

TensorFlow Version : 1.10.0
Python interpreter and version: 3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 17:14:51) 
[GCC 7.2.0]


In [2]:
dsroot = os.path.join(os.path.expanduser('~'),'datasets','kaggle-kepler')
os.listdir(dsroot)

['exoTest.csv', 'kepler-labelled-time-series-data.zip', 'exoTrain.csv']

In [3]:
import pandas as pd
train = pd.read_csv(os.path.join(dsroot,'exoTrain.csv'))
test = pd.read_csv(os.path.join(dsroot,'exoTest.csv'))
print('Training data\n',train.head())
print('Test data\n',test.head())

Training data
    LABEL   FLUX.1   FLUX.2   FLUX.3   FLUX.4   FLUX.5   FLUX.6  FLUX.7  \
0      2    93.85    83.81    20.10   -26.98   -39.56  -124.71 -135.18   
1      2   -38.88   -33.83   -58.54   -40.09   -79.31   -72.81  -86.55   
2      2   532.64   535.92   513.73   496.92   456.45   466.00  464.50   
3      2   326.52   347.39   302.35   298.13   317.74   312.70  322.33   
4      2 -1107.21 -1112.59 -1118.95 -1095.10 -1057.55 -1034.48 -998.34   

    FLUX.8  FLUX.9    ...      FLUX.3188  FLUX.3189  FLUX.3190  FLUX.3191  \
0   -96.27  -79.89    ...         -78.07    -102.15    -102.15      25.13   
1   -85.33  -83.97    ...          -3.28     -32.21     -32.21     -24.89   
2   486.39  436.56    ...         -71.69      13.31      13.31     -29.89   
3   311.31  312.42    ...           5.71      -3.73      -3.73      30.05   
4 -1022.71 -989.57    ...        -594.37    -401.66    -401.66    -357.24   

   FLUX.3192  FLUX.3193  FLUX.3194  FLUX.3195  FLUX.3196  FLUX.3197  
0      

In [4]:
x_train = train.drop('LABEL', axis=1)
y_train = train.LABEL-1 #subtract one because of TGBT
x_test = test.drop('LABEL', axis=1)
y_test = test.LABEL-1
numeric_column_headers = x_train.columns.values.tolist()

bucketized_features = [tf.feature_column.bucketized_column(source_column=tf.feature_column.numeric_column(key = column), 
                                                           boundaries=[x_train[column].mean()]) 
                       for column in numeric_column_headers]
all_features = bucketized_features  # since there are no other kind of features

In [5]:
batch_size = 32

train_input_fn = tf.estimator.inputs.pandas_input_fn(x = x_train,
                                                        y = y_train,
                                                        batch_size = batch_size,
                                                        shuffle = True,
                                                        num_epochs = None)
eval_input_fn = tf.estimator.inputs.pandas_input_fn(x = x_test,
                                                    y = y_test,
                                                    batch_size = batch_size,
                                                    shuffle = False,
                                                    num_epochs = 1)

In [6]:
n_trees = 100
n_steps = 100
model = tf.estimator.BoostedTreesClassifier(feature_columns=all_features, 
                                                 n_trees = n_trees,
                                                 n_batches_per_layer = batch_size,
                                                 model_dir='./tfbtmodel')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './tfbtmodel', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdd48c93b38>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [7]:
model.train(input_fn=train_input_fn, steps=n_steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tfbtmodel/model.ckpt-19201
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Saving checkpoints for 19201 into ./tfbtmodel/model.ckpt.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:loss = 1.0475121e-05, step = 19201
INFO:tensorflow:Sa

In [8]:
results = model.evaluate(input_fn=eval_input_fn)

for key,value in sorted(results.items()):
  print('{}: {}'.format(key, value))


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-06-14:58:44
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tfbtmodel/model.ckpt-19202
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-06-14:59:01
INFO:tensorflow:Saving dict for global step 19202: accuracy = 0.99122804, accuracy_baseline = 0.99122804, auc = 0.49911517, auc_precision_recall = 0.004386465, average_loss = 0.09851996, global_step = 19202, label/mean = 0.00877193, loss = 0.09749381, precision = 0.0, prediction/mean = 4.402521e-05, recall = 0.0
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 19202: ./tfbtmodel/model.ckpt-19202
accuracy: 0.99122804